# Distributional Equivalence Tests

In [ ]:
#%pip install openturns

     ---------------------------------------- 57.9/57.9 MB 2.2 MB/s eta 0:00:00
     -------------------------------------- 119.4/119.4 kB 1.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import openturns as ot
import openturns.viewer as viewer
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

ot.Log.Show(ot.Log.NONE)

In [3]:
D_mixed = pd.read_csv('./Data/FINAL_DATASET_G12.csv', index_col=0) 
D_mixed

,ipc_all,lag_1,lag_2,lag_3,lag_6,lag_9,lag_12,beef_lp,beef_cb,beef_sc,...,inflacion en bolivia,inflación bolivia,ipc,la inflacion,la inflación,pib,pib bolivia,que es inflacion,que es pib,freq
2010-01-01,NaN,0.263790,0.457498,0.784931,1.447299,5.323961,11.041182,16.000000,17.000000,15.650000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,day
2010-01-02,NaN,0.263790,0.457498,0.784931,1.447299,5.323961,11.041182,16.000000,16.700000,15.690000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,day
2010-01-03,NaN,0.263790,0.457498,0.784931,1.447299,5.323961,11.041182,16.000000,16.700000,15.690000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,day
2010-01-04,NaN,0.263790,0.457498,0.784931,1.447299,5.323961,11.041182,16.000000,16.700000,15.690000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,day
2010-01-05,NaN,0.263790,0.457498,0.784931,1.447299,5.323961,11.041182,16.000000,16.700000,15.690000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-28,NaN,6.182067,5.192869,3.953707,3.460018,1.863150,2.075432,29.000000,34.500000,24.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,day
2024-10-29,NaN,6.182067,5.192869,3.953707,3.460018,1.863150,2.075432,30.000000,34.500000,24.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,day
2024-10-30,NaN,6.182067,5.192869,3.953707,3.460018,1.863150,2.075432,30.670000,34.000000,24.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,day
2024-10-31,7.938964,6.182067,5.192869,3.953707,3.460018,1.863150,2.075432,28.524194,25.741935,24.201613,...,93.0,51.0,55.0,35.0,57.0,42.0,29.0,60.0,39.0,month


In [24]:
df_m = D_mixed.copy().query('freq == "month"')
df_m.drop(columns=['freq'], inplace=True)
df_m = df_m.loc[:, 'beef_lp':]

df_w = D_mixed.copy().query('freq == "week"')
df_w.drop(columns=['freq'], inplace=True)
df_w = df_w.loc[:, 'beef_lp':]

df_d = D_mixed.copy().query('freq == "day"')
df_d.drop(columns=['freq'], inplace=True)
df_d = df_d.loc[:, 'beef_lp':'libor']

df_md = df_m.loc[:, 'beef_lp':'libor']
df_mw = df_m.loc['2018-12-31':, ]

df_m.shape, df_w.shape, df_d.shape, df_md.shape, df_mw.shape

((178, 453), (305, 453), (5418, 436), (178, 436), (71, 453))

In [21]:
sample1 = df_md['beef_lp'].values
sample2 = df_d['beef_lp'].values

stats.kstest(sample1, sample2)

KstestResult(statistic=0.03735571399537953, pvalue=0.9621669767707768)

In [31]:
results_daily = {}
for column in df_md.columns:
    sample1 = df_md[column].values
    sample2 = df_d[column].values
    results_daily[column] = stats.kstest(sample1, sample2)

    results_daily_df = pd.DataFrame.from_dict(results_daily, orient='index').reset_index()
    results_daily_df.columns = ['feature', 'statistic', 'pvalue']
    results_daily_df.loc[:, ['statistic', 'pvalue']] = results_daily_df.loc[:, ['statistic', 'pvalue']].round(3)

results_daily_df

,feature,statistic,pvalue
0,beef_lp,0.037,0.962
1,beef_cb,0.045,0.858
2,beef_sc,0.035,0.981
3,beef_or,0.041,0.927
4,beef_po,0.029,0.998
...,...,...,...
431,soy_flour,0.035,0.977
432,soy_oil,0.019,1.000
433,lead,0.038,0.958
434,copper,0.024,1.000


In [32]:
results_daily_df.to_excel('./ks_test_daily.xlsx', index=False)

In [34]:
results_weekly = {}
for column in df_md.columns:
    sample1 = df_md[column].values
    sample2 = df_d[column].values
    results_weekly[column] = stats.kstest(sample1, sample2)

    results_weekly_df = pd.DataFrame.from_dict(results_weekly, orient='index').reset_index()
    results_weekly_df.columns = ['feature', 'statistic', 'pvalue']
    results_weekly_df.loc[:, ['statistic', 'pvalue']] = results_weekly_df.loc[:, ['statistic', 'pvalue']].round(3)

results_weekly_df

,feature,statistic,pvalue
0,beef_lp,0.037,0.962
1,beef_cb,0.045,0.858
2,beef_sc,0.035,0.981
3,beef_or,0.041,0.927
4,beef_po,0.029,0.998
...,...,...,...
431,soy_flour,0.035,0.977
432,soy_oil,0.019,1.000
433,lead,0.038,0.958
434,copper,0.024,1.000


In [35]:
results_weekly_df.to_excel('./ks_test_weekly.xlsx', index=False)